In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Step 1: Install gdown
!pip install -q gdown

# Step 2: Download the RCB.zip file using its file ID
!gdown "https://drive.google.com/file/d/1IrrswV9bXz6jjz8SUEVfBY6ILRJ17-xy/view?usp=sharing" --output RCB.zip


/usr/local/lib/python3.11/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=1IrrswV9bXz6jjz8SUEVfBY6ILRJ17-xy
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/1IrrswV9bXz6jjz8SUEVfBY6ILRJ17-xy/view?usp=sharing
To: /content/RCB.zip
94.6kB [00:00, 33.3MB/s]


In [ ]:
# Step 1: Install gdown if not already installed
!pip install -q gdown

# Step 2: Download the RCB.zip file correctly
!gdown --id 1IrrswV9bXz6jjz8SUEVfBY6ILRJ17-xy

# Step 3: Unzip it properly
import zipfile

with zipfile.ZipFile("RCB.zip", 'r') as zip_ref:
    zip_ref.extractall("rcb_dataset")  # Extract to "rcb_dataset" folder

print("✅ Extraction completed!")



/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1IrrswV9bXz6jjz8SUEVfBY6ILRJ17-xy
From (redirected): https://drive.google.com/uc?id=1IrrswV9bXz6jjz8SUEVfBY6ILRJ17-xy&confirm=t&uuid=f87cdc77-6a58-490e-92e4-2d5ec8311546
To: /content/RCB.zip
100% 266M/266M [00:04<00:00, 63.3MB/s]
✅ Extraction completed!


In [ ]:
# Step 3: Extract the ZIP contents
import zipfile

with zipfile.ZipFile("RCB.zip", 'r') as zip_ref:
    zip_ref.extractall("rcb_dataset")


In [ ]:
import os

# Check top-level folder after extraction
print("Top-level folders:", os.listdir("rcb_dataset"))

# Check subfolders inside 'RCB'
print("Subfolders in 'RRCCBB':", os.listdir("rcb_dataset/RCB"))


Top-level folders: ['RCB', '__MACOSX']
Subfolders in 'RRCCBB': ['non_offensive', 'offensive', '.DS_Store']


In [ ]:
!pip install pytesseract
import pytesseract
from PIL import Image

# OCR with Kannada + English
def extract_text(image_path):
    image = Image.open(image_path).convert("RGB")
    return pytesseract.image_to_string(image, lang='kan+eng')



In [ ]:
def clean_text(text):
    # KEEP Kannada characters, English alphabets, numbers, symbols
    cleaned = re.sub(r'[^A-Za-z0-9ಅ-ಹ಼ಽಾ-ೌೃ-ೄ೦-೯?!.,#*@ ]+', '', text)
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    return cleaned

In [ ]:
from transformers import AutoTokenizer

# Load a multilingual BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

def tokenize_text(text, max_length=64):
    return tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
# Full test using one meme
image_path = "/content/rcb_dataset/RCB/non_offensive/1b9.png"  # Replace with actual file
import re

def extract_and_tokenize(image_path):
    image = Image.open(image_path).convert("RGB")
    raw_text = pytesseract.image_to_string(image, lang='kan+eng')
    cleaned = clean_text(raw_text)
    tokens = tokenize_text(cleaned)
    return {
        "raw_text": raw_text,
        "cleaned_text": cleaned,
        "input_ids": tokens['input_ids'],
        "attention_mask": tokens['attention_mask']
    }

output = extract_and_tokenize(image_path)
print("Raw:", output["raw_text"])
print("Cleaned:", output["cleaned_text"])
print("Input IDs shape:", output["input_ids"].shape)


Raw: RCB after winning yesterday's match-

Ab bas 2 match jitne hai Virat aur, Maxwell bhi
minimum, phir hum playoffs | form mein aagye hain toh
ke Ze ell karjayenge Piao Se Decow ara areyS

 

Finals mein Virat bhai ~
finally, century maar, BeIFakelS (shi ; mg
dur hum final jeet jayenge. ,EesSald en) Netty

Cleaned: RCB after winning yesterdays matchAb bas 2 match jitne hai Virat aur, Maxwell bhiminimum, phir hum playoffs form mein aagye hain tohke Ze ell karjayenge Piao Se Decow ara areyS Finals mein Virat bhai finally, century maar, BeIFakelS shi mgdur hum final jeet jayenge. ,EesSald en Netty
Input IDs shape: torch.Size([1, 64])


In [ ]:
import os
import torch
import pytesseract
from PIL import Image
from transformers import AutoTokenizer
import re

# Setup
base_path = "/content/rcb_dataset/RCB"
label_map = {"offensive": 1, "non_offensive": 0}

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

# Clean text
def clean_text(text):
    text = re.sub(r'[^A-Za-z0-9ಅ-ಹ಼ಽಾ-ೌೃ-ೄ೦-೯?!., ]+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Tokenize with attention mask
def tokenize_text(text, max_length=64):
    return tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

# Collect all processed samples
processed_samples = []

for label_name, label_value in label_map.items():
    folder = os.path.join(base_path, label_name)
    for filename in os.listdir(folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(folder, filename)

            try:
                # Step 1: OCR
                image = Image.open(image_path).convert("RGB")
                raw_text = pytesseract.image_to_string(image, lang='kan+eng')

                # Step 2: Clean
                cleaned = clean_text(raw_text)

                # Step 3: Tokenize
                tokenized = tokenize_text(cleaned)

                sample = {
                    "image_path": image_path,
                    "raw_text": raw_text,
                    "cleaned_text": cleaned,
                    "input_ids": tokenized["input_ids"].squeeze(0),             # Tensor [64]
                    "attention_mask": tokenized["attention_mask"].squeeze(0),   # Tensor [64]
                    "label": label_value
                }

                processed_samples.append(sample)

            except Exception as e:
                print(f"Error processing {image_path}: {e}")


In [ ]:
torch.save(processed_samples, "text_features.pt")


In [ ]:
from google.colab import files
files.download("text_features.pt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import torch

# Step 1: Load the data first
text_data = torch.load("text_features.pt")  # <- Load the real object, not string

# Step 2: Then count samples
print(f"✅ Total samples in dataset: {len(text_data)}")


✅ Total samples in dataset: 758


In [ ]:
for i, sample in enumerate(processed_samples[:3]):
    print(f"\n📌 Sample {i+1}")
    print("-" * 50)
    print("🖼️ Image Path     :", sample["image_path"])
    print("📝 Raw Text       :", sample["raw_text"])
    print("🧹 Cleaned Text   :", sample["cleaned_text"])
    print("🆔 Input IDs      :", sample["input_ids"].tolist())
    print("🎯 Attention Mask :", sample["attention_mask"].tolist())
    print("🏷️ Label          :", "Offensive" if sample["label"] == 1 else "Non-Offensive")



📌 Sample 1
--------------------------------------------------
🖼️ Image Path     : /content/rcb_dataset/RCB/offensive/IMG_2927.PNG
📝 Raw Text       : NU Ere taays
ie

ea 4

ea

ON THE RCB WEBSITE & APP

    
 

TLS RO TILL et em lel) se
YL LY OLS ALO) LO LP
Ve CEC Mey a amore hee OL ETL

eT aR Rove AL Merl ae otle aie leks

LY eee

🧹 Cleaned Text   : NU Ere taaysieea 4eaON THE RCB WEBSITE APP TLS RO TILL et em lel seYL LY OLS ALO LO LPVe CEC Mey a amore hee OL ETLeT aR Rove AL Merl ae otle aie leksLY eee
🆔 Input IDs      : [101, 151, 12022, 10915, 10112, 11057, 13998, 17408, 11233, 69287, 10113, 30118, 17443, 40089, 11274, 88839, 44802, 29833, 30909, 11127, 157, 66934, 155, 11403, 157, 38850, 11369, 10131, 10266, 10141, 10161, 10126, 14703, 11369, 149, 14703, 19768, 10731, 47271, 11403, 149, 11403, 17521, 11779, 10112, 31078, 10858, 42110, 169, 23219, 100415, 19768, 74568, 11369, 10112, 11090, 169, 11273, 155, 21175, 47271, 10734, 10161, 102]
🎯 Attention Mask : [1, 1, 1, 1, 1, 1, 1, 1

 LSTM Text Encoder Module

In [ ]:
import torch

# Load your preprocessed text data (.pt file from earlier step)
text_data = torch.load("text_features.pt")



In [ ]:
import torch.nn as nn

class TextEncoderLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim=300, hidden_dim=256):
        super(TextEncoderLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)

    def forward(self, input_ids):
        x = self.embedding(input_ids)         # [batch, seq_len, embed_dim]
        _, (hidden, _) = self.lstm(x)         # hidden: [1, batch, hidden_dim]
        return hidden.squeeze(0)              # [batch, hidden_dim]


In [ ]:
from torch.utils.data import Dataset, DataLoader

class TextOnlyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, idx):
        return {
            "input_ids": self.data[idx]["input_ids"],
            "label": torch.tensor(self.data[idx]["label"], dtype=torch.float32)
        }

    def __len__(self):
        return len(self.data)

# Create dataset
text_dataset = TextOnlyDataset(text_data)

# Create DataLoader
text_loader = DataLoader(text_dataset, batch_size=16, shuffle=True)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Replace this with your actual tokenizer's vocab size
vocab_size = 119547  # for bert-base-multilingual-cased
text_encoder = TextEncoderLSTM(vocab_size=vocab_size).to(device)


In [ ]:
for batch in text_loader:
    input_ids = batch["input_ids"].to(device)         # [batch_size, seq_len]
    labels = batch["label"].unsqueeze(1).to(device)   # [batch_size, 1]

    # Forward pass
    text_embedding = text_encoder(input_ids)          # [batch_size, hidden_dim]
    print("Text Embedding Shape:", text_embedding.shape)  # e.g., [16, 256]
    break


Text Embedding Shape: torch.Size([16, 256])


In [ ]:
# Collect outputs
all_text_embeddings = []

for batch in text_loader:
    input_ids = batch["input_ids"].to(device)
    text_embedding = text_encoder(input_ids)
    all_text_embeddings.append(text_embedding.cpu())  # detach & move to CPU

# Stack into one tensor
all_text_embeddings = torch.cat(all_text_embeddings, dim=0)
torch.save(all_text_embeddings, "text_embeddings.pt")


In [ ]:
text_embeddings = torch.load("text_embeddings.pt")


In [ ]:
# Load your text encoder and move to eval mode
text_encoder.eval()
text_embeddings = []

with torch.no_grad():
    for batch in text_loader:
        input_ids = batch["input_ids"].to(device)
        text_embed = text_encoder(input_ids)  # shape: [batch_size, 256]
        text_embeddings.append(text_embed.cpu())  # move to CPU

# Stack into single tensor
all_text_embeddings = torch.cat(text_embeddings, dim=0)  # shape: [num_samples, 256]

# Save to file
torch.save(all_text_embeddings, "text_lstm_embeddings.pt")


In [ ]:
from google.colab import files
files.download("text_lstm_embeddings.pt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
for i in range(3):
    print(f"📌 Sample {i+1} - Text Embedding Vector (256-dim):\n")
    print(all_text_embeddings[i].numpy().round(3))  # round for readability
    print("\n" + "-"*80 + "\n")
    +


📌 Sample 1 - Text Embedding Vector (256-dim):

[ 0.033  0.007 -0.016 -0.013  0.024 -0.009  0.011  0.     0.007 -0.024
 -0.014  0.026 -0.039 -0.033 -0.02  -0.051 -0.032 -0.019 -0.003 -0.024
  0.026  0.012 -0.041  0.016 -0.038  0.017 -0.011 -0.031 -0.025  0.002
  0.028 -0.008 -0.031 -0.039 -0.001  0.03  -0.028  0.006 -0.029  0.006
 -0.022  0.004 -0.039 -0.013 -0.035  0.034  0.002  0.025  0.002 -0.02
 -0.004  0.012 -0.013  0.056  0.017  0.035 -0.069 -0.     0.011 -0.016
 -0.056  0.004 -0.02   0.033 -0.029 -0.009 -0.027 -0.048 -0.032  0.037
 -0.012 -0.046  0.018 -0.001  0.04   0.039 -0.062 -0.014 -0.007 -0.012
  0.046  0.041 -0.    -0.011 -0.022 -0.019  0.     0.021 -0.005 -0.005
  0.013  0.041  0.014  0.019 -0.021  0.002 -0.     0.024 -0.036  0.066
 -0.009 -0.014  0.009  0.018  0.02   0.015 -0.019  0.031 -0.004 -0.054
  0.027 -0.016  0.013  0.013 -0.046 -0.033  0.001  0.029 -0.005  0.018
 -0.038  0.038  0.002 -0.034  0.01   0.026  0.032 -0.04   0.04  -0.005
  0.02   0.023  0.012  0.025  0

In [ ]:
print("Total text samples:", len(text_data))


Total text samples: 758


In [ ]:
text_embeddings = torch.load("text_lstm_embeddings.pt")
print("Total embeddings generated:", text_embeddings.shape[0])


Total embeddings generated: 758


In [ ]:
from google.colab import drive
drive.mount('/content/drive')